In [128]:
import requests
import json
import pandas as  pd
# Running flask app from powershell
# $env:FLASK_APP = "api"
# $env:FLASK_ENV = "development"
# flask run

In [152]:
url = 'http://127.0.0.1:5000/createUser'
headers = {'content-type': 'application/json'}
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [153]:
response.json()

{'signingKey_hex': 'ef63bec579443cb0918755b68bef0e6217bc5788d595b919dd733554607184f1',
 'traderId': '8',
 'verifyKey_hex': 'c20b50d827701ceccafb8466cbcb1b145c30759d85ceda82aefa43f1315dee8a'}

In [154]:
url = 'http://127.0.0.1:5000/createMarket'

# copy from response
sk = response.json()['signingKey_hex']
vk = response.json()['verifyKey_hex']
tId = response.json()['traderId']

content_makemarket = {"signingKey_hex": sk,
                    "traderId": tId, 
                    "verifyKey_hex": vk,
                    "marketRootId": 10, 
                    "marketBranchId": 1, 
                    "marketMin": 0,
                    "marketMax":1.1}

response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)

In [155]:
response.json()

{'checks': 'True',
 'marketBranchId': 1,
 'marketMax': 1.1,
 'marketMin': 0,
 'marketRootId': 10,
 'traderId': '8'}

In [145]:
tId

'6'

In [108]:
pd.DataFrame(response.json(), index=[0])


,checks,marketBranchId,marketMax,marketMin,marketRootId,traderId
0,True,1,1.1,0,5,3


In [123]:
# Create a trade
url = 'http://127.0.0.1:5000/createTrade'

content_maketrade = {"signingKey_hex": sk, "traderId": 3, "verifyKey_hex": vk,
  "marketRootId": 5, "marketBranchId": 1, "price": 0.5, "quantity":1}

response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)

In [112]:
response.json()

{'checks': '(True, True)',
 'marketBranchId': 1,
 'marketRootId': 5,
 'price': 0.5,
 'quantity': 1,
 'traderId': 3}

In [113]:
# Create a matched trade
content_maketrade['quantity'] = -1
response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)

In [114]:
response.json()

{'checks': '(True, True)',
 'marketBranchId': 1,
 'marketRootId': 5,
 'price': 0.5,
 'quantity': -1,
 'traderId': 3}

In [115]:
# Create many matched trades
for i in range(10):
    content_maketrade['quantity'] = -1
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    content_maketrade['quantity']=1


In [119]:
url = 'http://127.0.0.1:5000/createMarket'

for i in range(10):
    content_makemarket = {"signingKey_hex": sk,
      "traderId": 3, "verifyKey_hex": vk,
      "marketRootId": i, "marketBranchId": 1, "marketMin": 0, "marketMax":1}

    response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)
    print('market created:' + str(i))
    
    

market created:0
market created:1
market created:2
market created:3
market created:4
market created:5
market created:6
market created:7
market created:8
market created:9


In [121]:
response.json()

{'checks': 'True',
 'marketBranchId': 1,
 'marketMax': 1,
 'marketMin': 0,
 'marketRootId': 9,
 'traderId': 3}

In [126]:
# Add a matched trade (with 10 base markets to see how it copes)

content_maketrade['quantity']=1
response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
content_maketrade['quantity'] = -1
response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)


In [127]:
response.json()

{'checks': '(True, True)',
 'marketBranchId': 1,
 'marketRootId': 5,
 'price': 0.5,
 'quantity': -1,
 'traderId': 3}